In [15]:
df1.head()

,Label,Text,DocName,DocType,cleanText,Length,category_id
0,Claims,Adjudication Match Referrals to Claims Certa...,Adjudication.docx,Word,adjudication match referrals claims certain se...,49095.0,0
1,Claims,ALW_CHG_AMT at Claim Line Level =WDA_PRIMARY_A...,APCD Claims Calculation.docx,Word,alwchgamt claim line level wdaprimaryallowedam...,5481.0,0
2,Claims,Region NW Mandated APCD in place? Yes Voluntar...,APCD Questions.pptx,powerpoint,region nw mandated apcd place yes voluntary ap...,4057.0,0
3,Claims,APCD (All Payer Claims Database) Overview Th...,apcd-KPWA.docx,Word,apcd payer claims database overview washington...,2916.0,0
4,Claims,Claims Data Warehouse CDW 101 Module How to Pi...,CDW 101 How to Pick Tables.pptx,powerpoint,claims data warehouse cdw module pick views de...,2468.0,0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000,sublinear_tf=True, min_df=6,norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df1.cleanText).toarray() # Remaps the words in the text column of 
                                                  # data frame into features (superset of words) with an importance assigned 
                                                  # based on each words frequency in the document and across documents

labels = df1.category_id    

In [17]:
import numpy as np

from sklearn.manifold import TSNE

# We will use the entire dataset as it is small 
# Sampling a subset of our dataset because t-SNE is computationally expensive

SAMPLE_SIZE = int(len(features))

np.random.seed(0)
indices = np.random.choice(range(len(features)), size=SAMPLE_SIZE, replace=False)         
projected_features = TSNE(n_components=2, random_state=0).fit_transform(features[indices]) # Array of all projected features 

In [20]:
# Reference for Word2Vec with Classification Model:https://github.com/susanli2016/NLP-with-Python

In [30]:
df1['stemmedcleanText'].head()

0    adjud match referr claim certain servic might ...
1    alwchgamt claim line level wdaprimaryallowedam...
2    region nw mandat apcd place yes voluntari apcd...
3    apcd payer claim databas overview washington s...
4    claim data warehous cdw modul pick view decemb...
Name: stemmedcleanText, dtype: object

Text Summarization using gensim

In [36]:
df1['Text'][6]

'Claims Authorization Matching Logic (CAML) for Claims  Having CAML information for claims is very critical to several processes within Health Plan Operations. It will enable them to audit CAML rules more efficiently , prevent appointment leakage of members to outside providers and provide transaprency.  The three important CAML fields are;  Hierarchy ID - This is the product line (HMO, Medicare, PPOs, POS) and what attributes are required to match an authorization. Attributes are the claim and authorization to matc such as service date, provider number, place of service and are different with each hierarchy.  Priority Number - Within each hierarchy there are different priority levels. This number indicates that.  Criteria Number - Criteria within the priority levels and hierarchy establishes the criteria (rule) that the claim reads. The rule claims use to link to an authorization. It determines , if any, the additional components of a claim required to mach within hierarchy.  A new JU

In [37]:
# import summarize from gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
# convert text to string format
text=str(df1['Text'][6])
# Summarize the text with ratio 0.01 (1% of total words.)
summarize(text,ratio=0.1)

'Identifies what order the CAML rules are processed in and the outcome  EDW_JOB_SEQUENCE_NBR  INTEGER  Job sequence number  EDW_INSERT_DTM  timestamp  Date and time the row was created in the data warehouse.'

In [38]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [39]:
import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

def clean_text2(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    #text = text.lower()
    text = re.sub(r'\[.?\]', '.', text)
    text = re.sub(r'[\n\r\t]', '.', text)
    #text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'[%s]' % re.escape('!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'), '', text)
    
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'[\w\.-]+@([\w\.-]+)','',text) # remove email addresses from text

    # remove duplicates
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text) #remove duplicated words in row
    text = text.replace('\t', '.')
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)


In [40]:
df1['Text2'] = pd.DataFrame(df1.Text.apply(lambda x: clean_text2(x)))
#df1['Length']=df1['cleanText'].str.len()

In [41]:
df1['Text'][26]

"Treatment Plan  Plan a future visit\t2  Document a treatment visit\t3       Plan a future visit  Click , then click the location in Planned Treatments, Draft Treatments, or Today that you'd like to move it to.  Fill in any additional details for the treatment.  If you are planning a procedure to perform on multiple teeth, you can use commas or dashes in the Site field to efficiently enter multiple teeth at once. For example, if you're planning to do sealants on teeth 1-5, type 1-5 in the Site field for that procedure.  Click  Plan to send this information to a scheduler. The treatment moves from the Draft Treatments section to the Planned Treatments section.     If you know the treatment should be scheduled right away, you can add the findings directly to the Planned Treatments section instead. When finished, collapse the treatment to send the information to the scheduler.    Add treatment options  In the Draft Treatments section, click  Add Option or select an option template to crea

In [42]:
# Summarize the text with ratio 0.01 (1% of total words.)
summarize(str(df1['Text2'][26]),ratio=0.1)

'If need associate billing diagnosis procedures document one Visit Diagnoses section Treatment Plan Activity.\nThen click open Treatment Plan Billing Diagnoses window associate visit diagnosis procedure.\nIf last visit treatment click Complete Treatment Plan youve finished procedures plan.'

In [43]:
# based on keywords it looks like we could have benefited from Stemming# based on keywords it looks like we could have benefited from Stemming

print(keywords(text,ratio=0.01))

Reference: https://github.com/Apress/natural-language-processing-recipes

In [44]:
# Import the packages

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.luhn import LuhnSummarizer

In [45]:
# Install sumy

#!pip install sumy

# Import the packages

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.luhn import LuhnSummarizer 

# Extracting and summarizing
LANGUAGE = "english"
SENTENCES_COUNT = 10
parser = PlaintextParser.from_string(df1['Text'][26], Tokenizer(LANGUAGE))
summarizer = LsaSummarizer()
summarizer = LsaSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

If you are planning a procedure to perform on multiple teeth, you can use commas or dashes in the Site field to efficiently enter multiple teeth at once.
For example, if you're planning to do sealants on teeth 1-5, type 1-5 in the Site field for that procedure.
When finished, collapse the treatment to send the information to the scheduler.
You can only move the visit, not the individual procedure or the entire treatment.
If you need to associate a billing diagnosis with your procedures, document one in the Visit Diagnoses section of the Treatment Plan Activity.
Then click  to open the Treatment Plan - Billing Diagnoses window and associate the visit diagnosis with each procedure.
If the same diagnosis applies to all procedures in the visit, you can select the Associate the selected diagnosis with all procedures in this visit that have not been completed check box to save time.
PROPRIETARY INFORMATION - This item and its contents may not be accessed, used, modified, reproduced, performe

In [46]:
# Extracting and summarizing
LANGUAGE = "english"
SENTENCES_COUNT = 10
parser = PlaintextParser.from_string(df1['Text'][26], Tokenizer(LANGUAGE))
summarizer = LuhnSummarizer()
summarizer = LuhnSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

Treatment Plan  Plan a future visit	2  Document a treatment visit	3       Plan a future visit  Click , then click the location in Planned Treatments, Draft Treatments, or Today that you'd like to move it to.
The treatment moves from the Draft Treatments section to the Planned Treatments section.
Add treatment options  In the Draft Treatments section, click  Add Option or select an option template to create a different treatment plan and compare pricing options for a patient.
To add more findings that the treatment plan addresses, click  next to the finding and add it to the treatment.
To quickly reorganize your treatment plan, move procedures between visits or drag and drop visits to change their order.
Document a treatment visit  When the patient arrives for a scheduled treatment, open the visit and go to the Treatment Plan.
If you need to associate a billing diagnosis with your procedures, document one in the Visit Diagnoses section of the Treatment Plan Activity.
Then click  to open

In [47]:
df1['LemmatizedText2'] = pd.DataFrame(df1.cleanText.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])))

In [48]:
list(df1)

['Label',
 'Text',
 'DocName',
 'DocType',
 'cleanText',
 'Length',
 'category_id',
 'stemmedcleanText',
 'LemmatizedcleanText',
 'Text2',
 'LemmatizedText2']

In [49]:
#ValueError: input must have more than one sentence. Hence have to use original Text with puntuations (periods) 

text=str(df1['LemmatizedcleanText'][0])
# Summarize the text with ratio 0.01 (1% of total words.)
#summarize(text,ratio=0.01)

print(keywords(text,ratio=0.1))

adjudication match referral claim
workqueues
workqueue
configuration
configured
configuring
configure
configurable
useful
matching
batched
field
enter
entering
entered
setting
set
extension use
code
following
follow
follows
report
reporting
batch type
evaluated matched
rule
service
required
require
referred
refer
refers
create
created
creates
creating
day
date
record
work user
cob
table tapestry profile
crossover
secondary
requirement benefit
method
topic
allowed
allow
allows
determine
determination
determines
determining
determined
feature requires
evaluate
evaluates
evaluation
build
building
information automatic
open
coverage
member
select
selection
selecting
organization
screen
need
needed
save
saving
saved
specified
specify
specifying
patient
vendor
contract
contracting
export
exported
exporting
worked selected
want
schedule
scheduler
scheduled
provider
provides
provide
provided
epic
line
adjudicate
adjudicated
adjudicating
calculated
calculation
calculate
calculates
calculator
ca

In [50]:
stemmer=PorterStemmer()
df1['stemmedText'] = pd.DataFrame(df1.Text.apply(lambda x: stemmed_text(x)))

In [51]:
text=str(df1['stemmedText'][0])
print(keywords(text,ratio=0.01))

claim
claims
thi
use
uses
referr
referral
referrals
workqueu
workqueues
workqueue
field
fields
